### Run this after all requests have finished processing (check status at https://cds.climate.copernicus.eu/)

In [ ]:
proj_dir="/path/to/main_project_folder/" # edit this line

import cdsapi 
import os
import itertools
import json
import multiprocessing as mp
import sys
sys.path.append(proj_dir)
from project_utils import parameters as param
from project_utils import load_region
from project_utils import prepare_inputs
import importlib
importlib.reload(param)
importlib.reload(load_region)
importlib.reload(prepare_inputs)

region_list = ['northcentral_north_america', 
               'southcentral_north_america', 
               'southeastern_north_america', 
               'southwestern_europe', 
               'western_europe', 
               'central_europe', 
               'eastern_europe', 
               'northeastern_europe', 
               'northeastern_asia', 
               'southeastern_asia', 
               'northsouthern_south_america', 
               'southsouthern_south_america', 
               'southwestern_africa', 
               'southeastern_africa', 
               'southwestern_australia', 
               'southeastern_australia', 
               'west_texas',
               'east_texas',
              ]

variables = ["volumetric_soil_water_layer_1", 
             "2m_temperature", 
             "geopotential"]

var_abbrev = ["swvl1", 
              "t2m", 
              "z"]

def download_if_complete(f_name):
    new_client = cdsapi.Client(wait_until_complete=False, delete=False, quiet=True)
    result = cdsapi.api.Result(new_client, {'request_id':request_id_dict[f_name]})
    result.update()
    reply = result.reply
    if reply['state'] == 'completed':
        if not os.path.exists(f_name):
            result.download(f_name)
            
for region_str in region_list:
    print(region_str)
    for i, curr_var in enumerate(variables):
        curr_var_abbrev = var_abbrev[i]
        print(curr_var, curr_var_abbrev)

        path_out = proj_dir+"input_data_ERA5/"+region_str+"/hourly/"+curr_var_abbrev+"/"

        with open(path_out+region_str+'_'+curr_var_abbrev+'_missing.json', 'r') as fp:
            global request_id_dict
            request_id_dict = json.load(fp)

        if len(request_id_dict.keys()):
            with mp.Pool(len(request_id_dict.keys())) as p:
                _ = p.map(download_if_complete, request_id_dict.keys()) 
        else:
            print('no missing requests\n')